# Task 2.2 Sourcing data with API

## Import libraries and concatenate data

In [1]:
import pandas as pd 
import numpy as np
import os 
import requests
import json
from datetime import datetime

In [2]:
os.getcwd()

'/Users/mainframe/Documents/GitHub/citibike_nyc'

In [3]:
# Create a list with all files in the folder using a list compehension

folderpath = r"2 Data/Original Data" # make sure to put the 'r' in front
filepaths  = [os.path.join(folderpath, name) for name in os.listdir(folderpath)]

In [4]:
filepaths

['2 Data/Original Data/202208-citibike-tripdata_3.csv',
 '2 Data/Original Data/202207-citibike-tripdata_2.csv',
 '2 Data/Original Data/202207-citibike-tripdata_3.csv',
 '2 Data/Original Data/202208-citibike-tripdata_2.csv',
 '2 Data/Original Data/JC-202203-citibike-tripdata.csv',
 '2 Data/Original Data/202207-citibike-tripdata_1.csv',
 '2 Data/Original Data/202208-citibike-tripdata_1.csv',
 '2 Data/Original Data/202210-citibike-tripdata_1.csv',
 '2 Data/Original Data/202207-citibike-tripdata_4.csv',
 '2 Data/Original Data/JC-202201-citibike-tripdata.csv',
 '2 Data/Original Data/202208-citibike-tripdata_4.csv',
 '2 Data/Original Data/202203-citibike-tripdata_2.csv',
 '2 Data/Original Data/JC-202209-citibike-tripdata.csv',
 '2 Data/Original Data/202210-citibike-tripdata_2.csv',
 '2 Data/Original Data/202203-citibike-tripdata_1.csv',
 '2 Data/Original Data/202210-citibike-tripdata_3.csv',
 '2 Data/Original Data/JC-202211-citibike-tripdata.csv',
 '2 Data/Original Data/202204-citibike-tripd

In [5]:
# Read and concatenate all files simultaneously

dtype = {
    'ride_id': str,
    'rideable_type': 'category',
    'start_station_name': str,
    'start_station_id': str,
    'end_station_name': str,
    'end_station_id': str,
    'member_casual': 'category'
}

df = pd.concat(
    (pd.read_csv(f, dtype=dtype, parse_dates=['started_at', 'ended_at']) for f in filepaths),
    ignore_index=True
)

#### Initially I tried using: 

#### df = pd.concat((pd.read_csv(f) for f in filepaths), ignore_index = True) 

#### for the code to read and concatenate all of the citibike file simultaneously. However, this led to some inefficiencies. It was reading each file in filepaths with pd.read_csv(f) one at a time, and using up more memory. It also gave me 35 warnings essentially saying that pandas detected mixed data types in colummns 5 and 7, and suggested specifying my dtypes on import. 
#### This way was kind of like the method of moving from one residence to another where you just grab things indiscriminately and throw them into boxes, not labeling anything, before sealing them up. Then, when you're in your new place you have to guess what's inside each box (cleaning supplies? clothing? books?), and reorganize as you go.

#### The code I ended up using tells pandas explicitly which dtypes to use upon import. This clears up the mixed dtype confusion, parses the dates to datetime64 upon read, and stores the 'member_casual' dtype as CATEGORY, thus using less memory and speeding up the entire process. 
#### This way is similar to the method of moving where you label your boxes ahead of time and put dishes in the box labeled "dishes", and shoes in the box labeled "shoes". This way you skip all the confusion and quickly organize your rooms as you go. 

In [6]:
df.shape

(30734291, 13)

In [7]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,9D0DC440CB40CF8E,electric_bike,2022-08-27 13:56:47.728,2022-08-27 14:02:56.651,Flatbush Ave & Ocean Ave,3704.04,3 St & Prospect Park West,3865.05,40.663657,-73.963014,40.668132,-73.973638,casual
1,2214991DFBE5C4D7,electric_bike,2022-08-20 10:37:02.756,2022-08-20 10:45:56.631,Forsyth St\t& Grand St,5382.07,E 11 St & 1 Ave,5746.14,40.717798,-73.993161,40.729538,-73.984267,casual
2,20C5D469563B6337,classic_bike,2022-08-31 18:55:03.051,2022-08-31 19:03:37.344,Perry St & Bleecker St,5922.07,Grand St & Greene St,5500.02,40.735354,-74.004831,40.721700,-74.002381,member
3,3E8791885BC189D1,classic_bike,2022-08-02 08:05:00.250,2022-08-02 08:16:52.063,FDR Drive & E 35 St,6230.04,Grand Army Plaza & Central Park S,6839.10,40.744219,-73.971212,40.764397,-73.973715,member
4,8DBCBF98885106CB,electric_bike,2022-08-25 15:44:48.386,2022-08-25 15:55:39.691,E 40 St & 5 Ave,6474.11,Ave A & E 14 St,5779.11,40.752052,-73.982115,40.730311,-73.980472,member


In [8]:
df.dtypes

ride_id                       object
rideable_type                 object
started_at            datetime64[ns]
ended_at              datetime64[ns]
start_station_name            object
start_station_id              object
end_station_name              object
end_station_id                object
start_lat                    float64
start_lng                    float64
end_lat                      float64
end_lng                      float64
member_casual               category
dtype: object

## Get weather data using NOAA's API

In [9]:
# Define NOAA token 

Token = 'pIUjJCxyaGdpoVSqMKuVbLUHLSBBNZVB' 

In [10]:
# Get the API 

r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid=GHCND:USW00014732&startdate=2022-01-01&enddate=2022-12-31', headers={'token':Token})

In [11]:
# Load the api response as a json

d = json.loads(r.text)

In [12]:
d

{'metadata': {'resultset': {'offset': 1, 'count': 365, 'limit': 1000}},
 'results': [{'date': '2022-01-01T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attributes': 'H,,S,',
   'value': 116},
  {'date': '2022-01-02T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attributes': 'H,,S,',
   'value': 114},
  {'date': '2022-01-03T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attributes': 'H,,S,',
   'value': 14},
  {'date': '2022-01-04T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attributes': 'H,,S,',
   'value': -27},
  {'date': '2022-01-05T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attributes': 'H,,S,',
   'value': 32},
  {'date': '2022-01-06T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attributes': 'H,,S,',
   'value': 49},
  {'date': '2022-01-07T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attribut

In [13]:
# Secure all items in the response that correspond to TAVG

avg_temps = [item for item in d['results'] if item['datatype']=='TAVG']

In [14]:
# Get only the date field from all average temperature readings

dates_temp = [item['date'] for item in avg_temps]

In [15]:
# Get the temperature from all average temperature readings

temps = [item['value'] for item in avg_temps]

In [16]:
temps

[116,
 114,
 14,
 -27,
 32,
 49,
 7,
 -25,
 14,
 16,
 -54,
 -19,
 40,
 48,
 -67,
 -80,
 39,
 18,
 32,
 51,
 -60,
 -59,
 -7,
 -2,
 36,
 -23,
 -42,
 1,
 -48,
 -71,
 -34,
 -17,
 23,
 64,
 58,
 -28,
 -48,
 5,
 41,
 28,
 63,
 86,
 118,
 28,
 -43,
 -47,
 16,
 116,
 99,
 1,
 -26,
 41,
 56,
 144,
 11,
 8,
 -11,
 24,
 8,
 26,
 77,
 56,
 -19,
 31,
 96,
 182,
 87,
 32,
 40,
 72,
 51,
 -23,
 53,
 116,
 136,
 82,
 139,
 131,
 143,
 104,
 107,
 72,
 57,
 96,
 104,
 59,
 -15,
 -12,
 26,
 99,
 131,
 72,
 70,
 76,
 97,
 86,
 91,
 128,
 114,
 86,
 88,
 134,
 154,
 163,
 157,
 157,
 83,
 79,
 76,
 94,
 106,
 146,
 133,
 128,
 104,
 115,
 116,
 82,
 104,
 129,
 136,
 121,
 124,
 120,
 172,
 148,
 100,
 104,
 135,
 154,
 161,
 171,
 160,
 179,
 188,
 204,
 197,
 178,
 155,
 154,
 212,
 278,
 210,
 174,
 162,
 164,
 202,
 204,
 207,
 233,
 283,
 178,
 203,
 195,
 219,
 200,
 222,
 218,
 237,
 244,
 218,
 218,
 205,
 242,
 239,
 233,
 203,
 252,
 217,
 173,
 205,
 221,
 191,
 192,
 219,
 258,
 270,
 243,
 22

In [17]:
# Put the results in a dataframe

df_temp = pd.DataFrame()

In [18]:
# Get only date and cast it to date time; convert temperature from tenths of Celsius to normal Celsius

df_temp['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
df_temp['avgTemp'] = [float(v)/10.0 for v in temps]

In [19]:
df_temp.tail()

,date,avgTemp
360,2022-12-27,-0.7
361,2022-12-28,3.4
362,2022-12-29,6.4
363,2022-12-30,9.3
364,2022-12-31,8.2


In [20]:
df_temp.head()

,date,avgTemp
0,2022-01-01,11.6
1,2022-01-02,11.4
2,2022-01-03,1.4
3,2022-01-04,-2.7
4,2022-01-05,3.2


In [21]:
df_temp.dtypes

date       datetime64[ns]
avgTemp           float64
dtype: object

In [22]:
# Extract just the date from the datetime

df['date'] = df['started_at'].dt.normalize()

In [23]:
# Merge with weather data

df_merged = df.merge(df_temp, how = 'left', on = 'date', indicator = True)

In [24]:
df_merged.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,date,avgTemp,_merge
0,9D0DC440CB40CF8E,electric_bike,2022-08-27 13:56:47.728,2022-08-27 14:02:56.651,Flatbush Ave & Ocean Ave,3704.04,3 St & Prospect Park West,3865.05,40.663657,-73.963014,40.668132,-73.973638,casual,2022-08-27,27.8,both
1,2214991DFBE5C4D7,electric_bike,2022-08-20 10:37:02.756,2022-08-20 10:45:56.631,Forsyth St\t& Grand St,5382.07,E 11 St & 1 Ave,5746.14,40.717798,-73.993161,40.729538,-73.984267,casual,2022-08-20,27.9,both
2,20C5D469563B6337,classic_bike,2022-08-31 18:55:03.051,2022-08-31 19:03:37.344,Perry St & Bleecker St,5922.07,Grand St & Greene St,5500.02,40.735354,-74.004831,40.721700,-74.002381,member,2022-08-31,25.6,both
3,3E8791885BC189D1,classic_bike,2022-08-02 08:05:00.250,2022-08-02 08:16:52.063,FDR Drive & E 35 St,6230.04,Grand Army Plaza & Central Park S,6839.10,40.744219,-73.971212,40.764397,-73.973715,member,2022-08-02,26.4,both
4,8DBCBF98885106CB,electric_bike,2022-08-25 15:44:48.386,2022-08-25 15:55:39.691,E 40 St & 5 Ave,6474.11,Ave A & E 14 St,5779.11,40.752052,-73.982115,40.730311,-73.980472,member,2022-08-25,28.1,both


In [25]:
df_merged['_merge'].value_counts(dropna = False)

both          30733651
left_only          640
right_only           0
Name: _merge, dtype: int64

In [26]:
# Which dates were not found in weather?

missing_dates = df_merged.loc[df_merged['_merge'] == 'left_only', 'date'].unique()

In [27]:
print(sorted(missing_dates))

[numpy.datetime64('2021-01-30T00:00:00.000000000'), numpy.datetime64('2021-02-15T00:00:00.000000000'), numpy.datetime64('2021-03-11T00:00:00.000000000'), numpy.datetime64('2021-03-14T00:00:00.000000000'), numpy.datetime64('2021-03-31T00:00:00.000000000'), numpy.datetime64('2021-04-30T00:00:00.000000000'), numpy.datetime64('2021-06-04T00:00:00.000000000'), numpy.datetime64('2021-07-10T00:00:00.000000000'), numpy.datetime64('2021-07-16T00:00:00.000000000'), numpy.datetime64('2021-07-19T00:00:00.000000000'), numpy.datetime64('2021-07-22T00:00:00.000000000'), numpy.datetime64('2021-08-16T00:00:00.000000000'), numpy.datetime64('2021-09-05T00:00:00.000000000'), numpy.datetime64('2021-09-13T00:00:00.000000000'), numpy.datetime64('2021-10-27T00:00:00.000000000'), numpy.datetime64('2021-11-07T00:00:00.000000000'), numpy.datetime64('2021-11-09T00:00:00.000000000'), numpy.datetime64('2021-11-13T00:00:00.000000000'), numpy.datetime64('2021-11-16T00:00:00.000000000'), numpy.datetime64('2021-11-17T0

In [28]:
# All dates in weather data
print(sorted(df_temp['date'].unique()))

[numpy.datetime64('2022-01-01T00:00:00.000000000'), numpy.datetime64('2022-01-02T00:00:00.000000000'), numpy.datetime64('2022-01-03T00:00:00.000000000'), numpy.datetime64('2022-01-04T00:00:00.000000000'), numpy.datetime64('2022-01-05T00:00:00.000000000'), numpy.datetime64('2022-01-06T00:00:00.000000000'), numpy.datetime64('2022-01-07T00:00:00.000000000'), numpy.datetime64('2022-01-08T00:00:00.000000000'), numpy.datetime64('2022-01-09T00:00:00.000000000'), numpy.datetime64('2022-01-10T00:00:00.000000000'), numpy.datetime64('2022-01-11T00:00:00.000000000'), numpy.datetime64('2022-01-12T00:00:00.000000000'), numpy.datetime64('2022-01-13T00:00:00.000000000'), numpy.datetime64('2022-01-14T00:00:00.000000000'), numpy.datetime64('2022-01-15T00:00:00.000000000'), numpy.datetime64('2022-01-16T00:00:00.000000000'), numpy.datetime64('2022-01-17T00:00:00.000000000'), numpy.datetime64('2022-01-18T00:00:00.000000000'), numpy.datetime64('2022-01-19T00:00:00.000000000'), numpy.datetime64('2022-01-20T0

In [29]:
# See exact dates that don't belong

df_merged.loc[df_merged['_merge'] == 'left_only', 'date'].sort_values().unique()

array(['2021-01-30T00:00:00.000000000', '2021-02-15T00:00:00.000000000',
       '2021-03-11T00:00:00.000000000', '2021-03-14T00:00:00.000000000',
       '2021-03-31T00:00:00.000000000', '2021-04-30T00:00:00.000000000',
       '2021-06-04T00:00:00.000000000', '2021-07-10T00:00:00.000000000',
       '2021-07-16T00:00:00.000000000', '2021-07-19T00:00:00.000000000',
       '2021-07-22T00:00:00.000000000', '2021-08-16T00:00:00.000000000',
       '2021-09-05T00:00:00.000000000', '2021-09-13T00:00:00.000000000',
       '2021-10-27T00:00:00.000000000', '2021-11-07T00:00:00.000000000',
       '2021-11-09T00:00:00.000000000', '2021-11-13T00:00:00.000000000',
       '2021-11-16T00:00:00.000000000', '2021-11-17T00:00:00.000000000',
       '2021-11-18T00:00:00.000000000', '2021-11-21T00:00:00.000000000',
       '2021-11-22T00:00:00.000000000', '2021-12-02T00:00:00.000000000',
       '2021-12-04T00:00:00.000000000', '2021-12-06T00:00:00.000000000',
       '2021-12-08T00:00:00.000000000', '2021-12-11

In [30]:
# Count rows per year

df_merged['date'].dt.year.value_counts()

2022    30733651
2021         640
Name: date, dtype: int64

In [31]:
# Filter out 2021 rides

df_merged = df_merged[df_merged['date'].dt.year == 2022]

In [32]:
df_merged['_merge'].value_counts(dropna = False)

both          30733651
left_only            0
right_only           0
Name: _merge, dtype: int64

In [33]:
df_merged.shape

(30733651, 16)

In [34]:
df_merged.to_csv('nyc_data.csv')

In [35]:
# See what's eating the most space in df_merged

df_merged.memory_usage(deep = True).sort_values(ascending = False) / 1024**2 # In MB

start_station_name    2254.611308
end_station_name      2251.518058
ride_id               2139.622233
rideable_type         2033.813268
start_station_id      1874.106947
end_station_id        1871.882539
end_lat                234.479149
avgTemp                234.479149
date                   234.479149
end_lng                234.479149
Index                  234.479149
start_lng              234.479149
start_lat              234.479149
ended_at               234.479149
started_at             234.479149
_merge                  29.310182
member_casual           29.310014
dtype: float64

In [37]:
df_merged.describe()

,start_lat,start_lng,end_lat,end_lng,avgTemp
count,3.073365e+07,3.073365e+07,3.069446e+07,3.069446e+07,3.073365e+07
mean,4.074032e+01,-7.397715e+01,4.074010e+01,-7.397724e+01,1.713369e+01
std,3.842966e-02,2.776968e-02,3.836903e-02,2.775996e-02,8.552076e+00
min,4.062737e+01,-7.408896e+01,4.056000e+01,-7.428000e+01,-1.170000e+01
25%,4.071687e+01,-7.399456e+01,4.071625e+01,-7.399462e+01,1.140000e+01
50%,4.073902e+01,-7.398197e+01,4.073827e+01,-7.398205e+01,1.790000e+01
75%,4.076229e+01,-7.396035e+01,4.076201e+01,-7.396040e+01,2.420000e+01
max,4.088398e+01,-7.387859e+01,4.106000e+01,-7.375000e+01,3.130000e+01


In [38]:
# Downcast numerical column data types

float_cols = ['start_lat', 'start_lng', 'end_lat', 'end_lng', 'avgTemp']

for col in float_cols:
    df_merged[col] = pd.to_numeric(df_merged[col], downcast='float')

In [39]:
# Convert string-heavy columns to category data type

category_cols = [
    'start_station_name',
    'end_station_name',
    'rideable_type',
    'start_station_id',
    'end_station_id',
    'member_casual'
]

for col in category_cols:
    df_merged[col] = df_merged[col].astype('category')

In [40]:
# Remove _merge

df_merged.drop(columns=['_merge'], inplace=True)

In [41]:
# Check total memory usage (in GB)

df_merged.memory_usage(deep=True).sum() / 1024**3

3.8648311402648687

In [42]:
# Check space by column

df_merged.memory_usage(deep=True).sort_values(ascending=False) / 1024**2  # MB

ride_id               2139.622233
Index                  234.479149
started_at             234.479149
ended_at               234.479149
date                   234.479149
start_lat              117.239574
start_lng              117.239574
end_lat                117.239574
end_lng                117.239574
avgTemp                117.239574
end_station_name        58.820522
start_station_name      58.819745
end_station_id          58.795259
start_station_id        58.794652
rideable_type           29.310194
member_casual           29.310014
dtype: float64

In [43]:
# Export merged, downsized dataset

df_merged.to_csv('nyc_data.csv')

In [44]:
# Optimize df_temp data type

df_temp['avgTemp'] = pd.to_numeric(df_temp['avgTemp'], downcast='float')

In [45]:
df_temp.dtypes

date       datetime64[ns]
avgTemp           float32
dtype: object

In [46]:
# Export df_temp

df_temp.to_csv('nyc_weather.csv')